In [ ]:
from weavepy import *
from config import BDD_VERSION, MODEL_NAMES, CLIM_VARS, ENER_VARS, ENER_VARS_TECHNOS, COUNTRIES_LIST
sns.set_context("talk")
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Climate Widgets ---
wdg_climate_variable = widgets.Dropdown(
    options=CLIM_VARS,
    value="TA",
    description="Climate Variable:",
)

wdg_aggregation_frequency = widgets.Dropdown(
    options=["H", "D", "M", "Y"],
    value="D",
    description="Frequency:",
)

wdg_aggregation_function = widgets.Dropdown(
    options=["mean", "sum", "max", "min"],
    value="mean",
    description="Aggregation Function:",
)

# --- Energy Widgets ---
wdg_energy_variable = widgets.Dropdown(
    options=ENER_VARS,
    value="SPV",
    description="Energy Variable:",
)

wdg_techno = widgets.Dropdown(
    options=ENER_VARS_TECHNOS[wdg_energy_variable.value],
    value="60",
    description="Technology:",
)

# --- Event Detection ---
wdg_climate_variable_comparison = widgets.RadioButtons(
    options=["<", "<=", ">", ">="],
    value="<=",
    description="Climate Operator:",
)

wdg_climate_variable_threshold = widgets.FloatSlider(
    min=-30, max=50, step=0.5, value=5,
    description="Climate Threshold:",
)

wdg_energy_variable_comparison = widgets.RadioButtons(
    options=["<", "<=", ">", ">="],
    value="<",
    description="Energy Operator:",
)

wdg_energy_variable_threshold = widgets.FloatSlider(
    min=0, max=1, step=0.05, value=0.05,
    description="Energy Threshold (capacity factor):",
)

# --- Time Periods ---
wdg_historical_period = widgets.IntRangeSlider(
    value=[1990, 2025],
    min=1950, max=2025, step=1,
    description="Historical period:",
)

wdg_future_period = widgets.IntRangeSlider(
    value=[2015, 2100],
    min=2015, max=2100, step=1,
    description="Future period:",
)

# --- Country Selection ---
wdg_country = widgets.Dropdown(
    options=COUNTRIES_LIST,
    value=COUNTRIES_LIST[0],
    description="Country:",
)

# --- Models/Scenarios ---
wdg_models = widgets.SelectMultiple(
    options=MODEL_NAMES,
    description="Model:",
)

wdg_scenarios = widgets.SelectMultiple(
    options=["historical", "SP126", "SP245", "SP370", "SP585"],
    value=[],
    description="Scenarios:",
)

btn_reset = widgets.Button(
    description="Clear selection",
    button_style="warning"
)

# Reset function
def reset_all(b):
    wdg_models.value = ()       # clears models
    wdg_scenarios.value = ()    # clears scenarios
    # add more widgets here if needed

# --- Execution Button + Status ---
status_label = widgets.Label(value="Ready.")
execute_button = widgets.Button(description="Execute", button_style="success")

def mark_dirty(change):
    status_label.value = "⚠️ Parameters changed. Click Execute to update."

# Mark widgets as "dirty" when changed
for w in [
    wdg_climate_variable, wdg_aggregation_frequency, wdg_aggregation_function,
    wdg_energy_variable, wdg_techno, wdg_climate_variable_comparison, wdg_climate_variable_threshold,
    wdg_energy_variable_comparison, wdg_energy_variable_threshold, wdg_country,
    wdg_historical_period, wdg_future_period, wdg_models, wdg_scenarios
]:
    w.observe(mark_dirty, names="value")

def update_techno(change):
    new_energy = change["new"]
    wdg_techno.options = ENER_VARS_TECHNOS[new_energy]
    wdg_techno.value = ENER_VARS_TECHNOS[new_energy][0]

wdg_energy_variable.observe(update_techno, names="value")

def on_execute_clicked(b):
    clear_output(wait=True)
    display(ui, status_label)  # redraw widgets
    status_label.value = "⏳ Running..."

    climate_variable = wdg_climate_variable.value
    aggregation_frequency = wdg_aggregation_frequency.value
    aggregation_function = wdg_aggregation_function.value
    energy_variable = wdg_energy_variable.value
    techno = wdg_techno.value
    climate_variable_comparison = wdg_climate_variable_comparison.value
    climate_variable_threshold = wdg_climate_variable_threshold.value
    energy_variable_comparison = wdg_energy_variable_comparison.value
    energy_variable_threshold = wdg_energy_variable_threshold.value
    country = wdg_country.value
    historical_period = wdg_historical_period.value
    future_period = wdg_future_period.value
    models = wdg_models.value
    scenarios = wdg_scenarios.value
    
    data = load_vars([climate_variable, energy_variable], bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], models = models, scenarios = scenarios, 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True)
    
    climate_days = identify_pb_days(data[climate_variable], climate_variable_comparison, climate_variable_threshold).rename("climate_event")
    energy_days = identify_pb_days(data[energy_variable], energy_variable_comparison, energy_variable_threshold).rename("energy_event")
    compound_days = (climate_days & energy_days).rename("compound_event")

    climate_events = identify_events_whole_base(climate_days)
    energy_events = identify_events_whole_base(energy_days)
    compound_events = identify_events_whole_base(compound_days)

    if list(models) != []:
        climate_events = climate_events[climate_events['model'].isin(list(models))]
        energy_events = energy_events[energy_events['model'].isin(list(models))]
        compound_events = compound_events[compound_events['model'].isin(list(models))]
        climate_days = climate_days.sel(model=list(models))
        energy_days = energy_days.sel(model=list(models))
        compound_days = compound_days.sel(model=list(models))
    
    if list(scenarios) != []:
        climate_events = climate_events[climate_events['scenario'].isin(list(scenarios))]
        energy_events = energy_events[energy_events['scenario'].isin(list(scenarios))]
        compound_events = compound_events[compound_events['scenario'].isin(list(scenarios))]
        climate_days = climate_days.sel(scenario=list(scenarios))
        energy_days = energy_days.sel(scenario=list(scenarios))
        compound_days = compound_days.sel(scenario=list(scenarios))

    N_climate_events = climate_events.groupby(["model", "scenario", "country", "year"]).eventID.nunique().reset_index()
    N_energy_events = energy_events.groupby(["model", "scenario", "country", "year"]).eventID.nunique().reset_index()
    N_compound_events = compound_events.groupby(["model", "scenario", "country", "year"]).eventID.nunique().reset_index()

    nb_event_timeseries(N_climate_events, N_energy_events, N_compound_events)

    event_duration_hist(climate_events, energy_events, compound_events)

    event_seasonality_kde(climate_days, energy_days, compound_days)

    status_label.value = "✅ Done!"

execute_button.on_click(on_execute_clicked)
btn_reset.on_click(reset_all)

# --- Layout ---
ui = widgets.VBox([
        widgets.HBox([
            widgets.VBox([wdg_climate_variable],
                         #layout=widgets.Layout(justify_content="space-between", align_items="center", border="2px solid #888", padding="5px")
                         ),
            widgets.VBox([wdg_energy_variable, wdg_techno],
                         #layout=widgets.Layout(justify_content="space-between", align_items="center", border="2px solid #888", padding="5px")
                         ),],
            #layout=widgets.Layout(align_items="center", border="2px solid #888", padding="5px")
            ),
        widgets.VBox([
            widgets.HBox([wdg_aggregation_frequency, wdg_aggregation_function]),
            widgets.HBox([wdg_country]),
        ],#layout=widgets.Layout(justify_content="space-between", align_items="center", border="2px solid #888", padding="5px")
                     ),
    widgets.HBox([wdg_historical_period, wdg_future_period]),
    widgets.HBox([wdg_climate_variable_comparison, wdg_climate_variable_threshold]),
    widgets.HBox([wdg_energy_variable_comparison, wdg_energy_variable_threshold]),
    
    
    widgets.HBox([wdg_models, wdg_scenarios,btn_reset]),
    execute_button
])

display(ui, status_label)


Label(value='Ready.')